In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil

In [ ]:
""" 直接存到雲端 大約5.5GB 先確認雲端空間還夠
!wget -O MOT17.zip https://motchallenge.net/data/MOT17.zip
!unzip /content/MOT17.zip -d /content/drive/MyDrive/
"""

In [ ]:
path = '/content/drive/MyDrive/trainyolo'
train = ['02','04','09','10','11','13']
valid = ['12','14']

In [ ]:
os.makedirs(path, exist_ok=True)

In [ ]:
def copy_and_rename(endswith, source_dir, dest_dir, name):

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for filename in os.listdir(source_dir):
        if filename.endswith(endswith):
            source_file = os.path.join(source_dir, filename)
            dest_file = os.path.join(dest_dir, f"{name}_{filename}")
            shutil.copy(source_file, dest_file)

In [ ]:
def labels(label_file,output_label_dir):

    if not os.path.exists(output_label_dir):
        os.makedirs(output_label_dir)

    with open(label_file, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split(',')
        if len(parts) != 7:
            print(f"格式錯誤: {line.strip()}")
            continue

        frame = int(parts[0])  # frame
        obj_id = int(parts[1])  # id
        left = float(parts[2])  # left
        top = float(parts[3])  # top
        width = float(parts[4])  # width
        height = float(parts[5])  # height

        # 算center還有做normlization
        img_width = 1920  # pixel
        img_height = 1080  # pixel
        x_center = (left + width / 2 - 1) / img_width
        y_center = (top + height / 2) / img_height
        norm_width = width / img_width
        norm_height = height / img_height

        # 轉換為 YOLO 格式
        yolo_label = f"0 {x_center} {y_center} {norm_width} {norm_height}\n"
        output_label_path = os.path.join(output_label_dir, f"{frame:06}.txt")
        with open(output_label_path, 'a') as out_f:
            out_f.write(yolo_label)
    print(f"完成")

In [ ]:
for i in train:
    name = f'MOT17-{i}-FRCNN'
    source_dir = f'/content/drive/MyDrive/MOT17/train/{name}/img1'
    dest_dir = f'{path}/train/images/'
    copy_and_rename('.jpg', source_dir, dest_dir, name)

for i in valid:
    name = f'MOT17-{i}-FRCNN'
    source_dir = f'/content/drive/MyDrive/MOT17/test/{name}/img1'
    dest_dir = f'{path}/valid/images/'
    copy_and_rename('.jpg', source_dir, dest_dir, name)

In [ ]:
for i in train:
    name = f'MOT17-{i}-FRCNN'
    train_file = f'/content/drive/MyDrive/MOT17/train/{name}/det/det.txt'
    train_label_dir = f'/content/labels/{name}'
    labels(train_file,train_label_dir)

    source_dir = train_label_dir
    dest_dir = f'{path}/train/labels/'
    copy_and_rename('.txt', source_dir, dest_dir, name)

for i in valid:
    name = f'MOT17-{i}-FRCNN'
    valid_file = f'/content/drive/MyDrive/MOT17/test/{name}/det/det.txt'
    valid_label_dir = f'/content/labels/{name}'
    labels(train_file,train_label_dir)

    source_dir = train_label_dir
    dest_dir = f'{path}/valid/labels/'
    copy_and_rename('.txt', source_dir, dest_dir, name)